In [ ]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

In [ ]:
%%time
df = pd.read_csv("/content/drive/MyDrive/Major Project/dataset.csv", index_col = 'Index')

CPU times: user 40.1 s, sys: 5.78 s, total: 45.9 s
Wall time: 1min 8s


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5956842 entries, 0 to 5956841
Data columns (total 23 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   pH                      float64
 1   Iron                    float64
 2   Nitrate                 float64
 3   Chloride                float64
 4   Lead                    float64
 5   Zinc                    float64
 6   Color                   object 
 7   Turbidity               float64
 8   Fluoride                float64
 9   Copper                  float64
 10  Odor                    float64
 11  Sulfate                 float64
 12  Conductivity            float64
 13  Chlorine                float64
 14  Manganese               float64
 15  Total Dissolved Solids  float64
 16  Source                  object 
 17  Water Temperature       float64
 18  Air Temperature         float64
 19  Month                   object 
 20  Day                     float64
 21  Time of Day             float64
 22 

In [ ]:
df['Target'].value_counts()

,count
Target,
0,4151590
1,1805252


In [ ]:
len(df)

5956842

In [ ]:
# Firstly, we convert the data types to increase the computational speed as required
for label, content in df.items():
  if not pd.api.types.is_any_real_numeric_dtype(content):
    print(label)
    df[label] = content.astype('category').cat.as_ordered()
    df[label] = df[label].cat.codes
    print(df[label].value_counts())

Color
Color
 0    1787911
 3    1786234
 1    1079772
 2     758138
 4     539048
-1       5739
Name: count, dtype: int64
Source
Source
 6    734502
 1    734389
 7    734315
 0    733778
 3    733298
 4    732980
 5    732700
 2    732618
-1     88262
Name: count, dtype: int64
Month
Month
 7     498700
 5     498132
 8     498043
 4     497875
 2     497349
 1     497072
 10    496061
 0     482261
 6     482016
 11    481456
 9     481020
 3     451189
-1      95668
Name: count, dtype: int64


In [ ]:
def optimise_df(Df):
  for col in Df.columns:
    col_type = Df[col].dtype

    if col_type == 'float64' or col_type == 'float32':
      Df[col] = pd.to_numeric(Df[col], downcast = 'float')

    elif col_type == "int64" or col_type == "int32":
      Df[col] = pd.to_numeric(Df[col], downcast="integer")

  return Df

In [ ]:
df= optimise_df(df)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5956842 entries, 0 to 5956841
Data columns (total 23 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   pH                      float32
 1   Iron                    float32
 2   Nitrate                 float32
 3   Chloride                float32
 4   Lead                    float32
 5   Zinc                    float32
 6   Color                   int8   
 7   Turbidity               float32
 8   Fluoride                float32
 9   Copper                  float32
 10  Odor                    float32
 11  Sulfate                 float32
 12  Conductivity            float32
 13  Chlorine                float32
 14  Manganese               float32
 15  Total Dissolved Solids  float32
 16  Source                  int8   
 17  Water Temperature       float32
 18  Air Temperature         float32
 19  Month                   int8   
 20  Day                     float32
 21  Time of Day             float32
 22 

In [ ]:
df.isnull().sum()

,0
pH,116054
Iron,39753
Nitrate,105725
Chloride,175531
Lead,26909
Zinc,156126
Color,0
Turbidity,49815
Fluoride,189156
Copper,199402


In [ ]:
df = df.dropna()

In [ ]:
df.isnull().sum()

,0
pH,0
Iron,0
Nitrate,0
Chloride,0
Lead,0
Zinc,0
Color,0
Turbidity,0
Fluoride,0
Copper,0


In [ ]:
df.duplicated().sum()

110042

In [ ]:
df = df.drop_duplicates()

In [ ]:
len(df)

4001471

In [ ]:
def robust_scaling(DF):
  medians = DF.median()
  q1 = DF.quantile(0.25)
  q3 = DF.quantile(0.75)
  iqr = q3 -q1

  df_scaled = (DF - medians)/ iqr

  return df_scaled, medians,iqr

In [ ]:
df, Median, IQR = robust_scaling(df)

In [ ]:
# In this step, we will take care of the extreme outliers
df = df.clip(lower = df.quantile(0.05), upper = df.quantile(0.95), axis = 1)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4001471 entries, 1 to 5956840
Data columns (total 23 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   pH                      float64
 1   Iron                    float64
 2   Nitrate                 float64
 3   Chloride                float64
 4   Lead                    float64
 5   Zinc                    float64
 6   Color                   float64
 7   Turbidity               float64
 8   Fluoride                float64
 9   Copper                  float64
 10  Odor                    float64
 11  Sulfate                 float64
 12  Conductivity            float64
 13  Chlorine                float64
 14  Manganese               float64
 15  Total Dissolved Solids  float64
 16  Source                  float64
 17  Water Temperature       float64
 18  Air Temperature         float64
 19  Month                   float64
 20  Day                     float64
 21  Time of Day             float64
 22 

In [ ]:
df= optimise_df(df)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4001471 entries, 1 to 5956840
Data columns (total 23 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   pH                      float32
 1   Iron                    float32
 2   Nitrate                 float32
 3   Chloride                float32
 4   Lead                    float64
 5   Zinc                    float32
 6   Color                   float32
 7   Turbidity               float32
 8   Fluoride                float32
 9   Copper                  float32
 10  Odor                    float32
 11  Sulfate                 float32
 12  Conductivity            float32
 13  Chlorine                float32
 14  Manganese               float32
 15  Total Dissolved Solids  float32
 16  Source                  float32
 17  Water Temperature       float32
 18  Air Temperature         float32
 19  Month                   float32
 20  Day                     float32
 21  Time of Day             float32
 22 

In [ ]:
df.to_csv("/content/drive/MyDrive/Major Project/dataset_processed.csv")